# <center>第13章 Python建模库介绍

## 13.1 pandas与模型代码接口

In [ ]:
data=pd.DataFrame({'x1':[1,2,3,4,5],
                   'x2':[0.01,-0.01,0.25,-4.1,0],
                   'y':[chr(x) for x in np.random.randint(98,100,5)]})
data.values
data['y']=data['y'].astype('category')
dummy=pd.get_dummies(data['y'],prefix='category')
data.drop('y',axis=1,inplace=True)

## 13.2 用Patsy创建模型描述

### 1.Patsy简介

>Patsy是Python的一个库，使用简短的字符串“公式语法”描述统计模型（尤其是线性模型），可能是受到了R和S统计编程语言的公式语法的启发。Patsy适合描述statsmodels的线性模型。Patsy的公式是一个特殊的字符串语法
<font color='#AEEEEE'><center>y~x0+x1</center></font>

In [ ]:
import pandas as pd
import numpy as np
import patsy

data=pd.DataFrame({'x0':[1,2,3,4,5],
                   'x1':[0.01,-0.01,0.25,-4.1,0.1],
                   'y':[-1.5,0,3.6,1.3,-2]})
data
y,x=patsy.dmatrices('y~x0+x1',data)
#x,y是Patsy的DesignMatrix实例,这些实例是Numpy的ndarray
type(y)
y
x
#np.asarray: convert the input to an array
np.asarray(y)
np.asarray(x)
#x会有一个截距1，添加0到模型可以不显示intercept
patsy.dmatrices('y~0+x0+x1',data)
#Patsy对象可以直接传递到算法
#np.linalg.lstsq执行普通最小二乘法
#a是x的rank，b是x的特征值
coef,resid,a,b=np.linalg.lstsq(x,y)

### 2.用Patsy公式进行数据转换

In [ ]:
import pandas as pd
import numpy as np
import patsy

data=pd.DataFrame({'x0':[1,2,3,4,5],
                   'x1':[0.01,-0.01,0.25,-4.1,0.1],
                   'y':[-1.5,0,3.6,1.3,-2]})
y,x=patsy.dmatrices('y~x0+np.log(x0+1)+1',data)
#常见的变量转换包括标准化和中心化
y,x=patsy.dmatrices('y~standardize(x0)+center(x1)+1',data)

#拟合模型到另一个数据集
#patsy.build_design_matrices可以使用原始样本数据的处理信息来转换新数据
new_data=pd.DataFrame({'x0':[6,7,8,9],
                       'x1':[3.1,-0.5,0,2.3],
                       'y':[1,2,3,4]})
new_x=patsy.build_design_matrices([x.design_info],new_data)
#Patsy中的'+'不是加法的含义，把数据集相加时，用特殊的'I'函数封装起来
y1,x1=patsy.dmatrices('y~I(x0+x1)',new_data)

### 3.分类数据和Patsy

In [ ]:
import pandas as pd 
import numpy as np 
import patsy

#在Patsy中使用非数值数据，会默认转换为虚拟变量
data=pd.DataFrame({'key1':[chr(x) for x in np.random.randint(97,99,7)],
                   'key2':[0,1,1,0,0,1,1],
                   'v1':[1,2,3,4,5,6,7],
                   'v2':[-1,0,2.5,-0.5,4,-1.4,-1.2]})
y,x=patsy.dmatrices('v2~key1',data)
x
#使用C函数，数值列可以截取为分类量
y1,x1=patsy.dmatrices('v2~C(v1)',data)
np.array(x1)
#在分类模型中使用多个分类名，事情就会变复杂
data['key2']=data['key2'].map({0:'zero',1:'one'})
y1,x1=patsy.dmatrices('v2~key1+key2',data)
x1
# v1:v2的意思是v1*v2,如果把v1*v2写进公式,v1和v2也会列式出来
y2,x2=patsy.dmatrices('key1~v1:v2',data)
x2

## 13.3 statsmodels介绍

>statsmodes是python拟合统计模型、进行统计试验和数据探索可视化的库包含的模型有：
 1. 线型模型、广义线型模型和健壮线型模型；
 2. 线型混合效应模型方差(ANOVA)方法分析；
 3. 时间序列过程和状态空间模型；
 4. 广义矩估计
 
 
### 1.估计线性模型

In [ ]:
import pandas as pd
import numpy as np
import patsy
import statsmodels.api as sm
import statsmodels.formula.api as smf

#线性模型有两种不同的接口：基于数组和基于公式

#这个np.concatenate可以用np.c_代替
#这样：x=np.c_[dnorm(0,0.4,100),dnorm(0,0.6,100),dnorm(0,0.2,100)]
x=np.concatenate([np.random.randn(100,1),np.random.randn(100,1),np.random.randn(100,1)],axis=1)
eps=dnorm(0,0.1,100)
beta=np.array([0.1,0.3,0.5]).reshape((3,1))
y=np.dot(x,beta)+eps
y.shape
#sm_add_constant函数可以添加一个截距到现存的列
x_model=sm.add_constant(x)
x_model.shape
model=sm.OLS(y,x_model)
#模型的fit方法返回了一个回归结果对象，它包含估计模型参数和其他内容
result=model.fit()
result.params

#现在使用statsmodels的公式API和Patsy
data=pd.DataFrame(x,columns=['col1','col2','col3'])
data['y']=y
data.head()
result=smf.ols('y~1+col1+col2+col3',data=data).fit()
result.params
result.tvalues
result.pvalues
result.rsquared
#根据回归参数预测值
a=pd.DataFrame(np.random.randn(3,5),columns=['col1','col2','a','b','col3'])
result.predict(a)


### 2.估计时间序列

In [ ]:
import pandas as pd
import numpy as np
import random

x0=4
value=[x0,x0]
noise=np.random.randn(100)*0.1
for i in np.arange(100):
    x=value[-1]*0.8+value[-2]*(-0.4)+noise[i]
    value.append(x)

#这个数据有两个延迟(也就是AR(2))
#由于不知道滞后量的个数，先用多的来拟合
maxlag=5
model=sm.tsa.AR(value)
result=model.fit(maxlag)
result.params
result.summary()

## 13.4 scikit-learn介绍

>scikit-learn是一个广泛使用、用途多样的Python机器学习库

In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression

path=r"D:\文档\Python Scripts\data_test\titanic.csv"
train=pd.read_csv(path)
train.head()
#检查缺失值
train.isnull().astype(int).sum()
train['Age']=train['Age'].fillna(train['Age'].median())
#增加IsFemale作为'Sex'列的编码
train['IsFemale']=(train['Sex']=='female').astype(int)
predictor=['Pclass','IsFemale','Age']
model=LogisticRegression()
model.fit(train[predictor],train['Survived'])
#预测
model.predict(?)

#交叉验证通过分割训练数据来模拟样本外预测
from sklearn.linear_model import LogisticRegressionCV

model_cv=LogisticRegressionCV(10)
model_cv.fit(train[predictor],train['Survived'])

#手动交叉验证，使用cross_val_scroe处理数据分割
from sklearn.model_selection import cross_val_score

model=LogisticRegression(C=10)
scores=cross_val_score(model,train[predictor],train['Survived'],cv=4)
scores